🚨 Ten activada la **Aceleración por hardware** con GPU en `"Entorno de ejecución" > "Cambiar tipo de entorno de ejecución"`

In [ ]:
#@title ⚙️ Ejecutar esta celda para instalar las librería.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install ffmpeg

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-s2icwxsr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-s2icwxsr
  Resolved https://github.com/openai/whisper.git to commit b91c907694f96a3fb9da03d4bbdc83fbcd3a40a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=eddb3bc1a1202f950db9ef608ef43f9c0ceb20f399d078a9fe7fb2bfe0d3fca4
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 🎙️ **Grabar y transcribir** [Elige la tarea]

In [ ]:
Tarea = "Transcript to Language" #@param ["Transcript to Language", "Translate to English"]
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output

task = "translate" if Tarea == "Translate to English" else "transcribe"

audio, sr = get_audio()
write('record.wav', sr, audio)

!whisper "record.wav" --task {task} --model medium --verbose False

clear_output()
if task == "translate":
  print("-- TRADUCCIÓN A INGLÉS --\n")
else:
  print("-- TRANSCRIPCIÓN A ESPAÑOL --\n")

print(open('record.wav.txt').read())

AttributeError: ignored

## 👉  Sólo **transcribir** audio al idioma original. (X ► X)

In [ ]:
!whisper "/content/voz-2.mp3" --task transcribe --model large --verbose False --output_dir audio_transcription

100%|██████████████████████████████████████| 2.87G/2.87G [00:28<00:00, 110MiB/s]
Detected language: Spanish
100% 8539/8539 [00:29<00:00, 291.55frames/s]


## 👉  Sólo **traducir** audio del idioma original al inglés. (X ► English)

In [ ]:
!whisper "/content/voz-2.mp3" --task translate --model medium

100%|█████████████████████████████████████| 1.42G/1.42G [00:42<00:00, 35.9MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Spanish
[00:00.000 --> 00:18.000]  The abbreviated natural language processing, PLN in English, Natural Language Processing, NLP is a field of the sciences of computer science, artificial intelligence and linguistics that studies interactions between computers and human language.
[00:18.000 --> 00:31.000]  It deals with the formulation and research of mechanisms efficiently computationally for communication between people and machines by means of natural language, that is, of the languages of the world.
[00:31.000 --> 00:47.000]  It does not deal with communication by means of natural languages in an abstract way, but to design mechanisms to communicate that are efficiently computationally, that can be done by means of programs that execute or simulate communication.
[00:47.000 --> 00:57.000]  T